In [33]:
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from yahoofinancials import YahooFinancials
import wikipedia as wp
from concurrent.futures import ThreadPoolExecutor



In [28]:
token = '39L3MfwUDvhFO5we7eODz_Z-uXdrlSmV48djjnfwbzYL0Wz9ZpHcej7MzgsSZCvh-qsqop2OpZ4SdVhidJj80g=='
org = "my-org"
url = "http://3.109.41.82:8086"
bucket='raf'

In [42]:
df= pd.read_csv('/home/mehedi/Documents/docker_influxdb/DTCC_data/extracted_files/Commodities/CFTC_CUMULATIVE_COMMODITIES_2023_07_25.csv')
df

,Dissemination Identifier,Original Dissemination Identifier,Action type,Event type,Event timestamp,Amendment indicator,Asset Class,Product name,Cleared,Mandatory clearing indicator,...,Option Style,Package indicator,Package transaction price,Package transaction price currency,Package transaction price notation,Package transaction spread,Package transaction spread currency,Package transaction spread notation,Physical delivery location-Leg 1,Delivery Type
0,697754337,696534327.0,MODI,TRAD,2023-03-23T18:31:09Z,True,CO,Commodity:Agricultural:GrainsOilSeeds:Exotic,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,697754338,696533897.0,MODI,TRAD,2023-04-04T18:32:09Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,697754339,696533431.0,MODI,TRAD,2023-03-24T20:40:31Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,697754340,696534323.0,MODI,TRAD,2023-04-04T18:32:32Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,697754341,696534325.0,MODI,TRAD,2023-04-13T16:55:39Z,True,CO,Commodity:Agricultural:Softs:Exotic,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7109,697721209,NaN,NEWT,TRAD,2023-07-25T00:28:03Z,False,CO,Commodity:Agricultural:GrainsOilSeeds:Swap:Cash,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7110,697721212,NaN,NEWT,TRAD,2023-07-25T00:30:41Z,NaN,CO,Commodity:Metals:NonPrecious:Swap:Cash,N,False,...,NaN,True,9.9999999999,NaN,3.0,NaN,NaN,NaN,NaN,Cash
7111,697721227,NaN,NEWT,TRAD,2023-07-25T00:28:23Z,False,CO,Commodity:Agricultural:GrainsOilSeeds:Swap:Cash,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7112,697721230,NaN,NEWT,TRAD,2023-07-25T00:28:25Z,False,CO,Commodity:Agricultural:GrainsOilSeeds:Swap:Cash,N,False,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns

Index(['Original Dissemination Identifier', 'Action type', 'Event type',
       'Event timestamp', 'Amendment indicator', 'Asset Class', 'Product name',
       'Cleared', 'Mandatory clearing indicator', 'Execution Timestamp',
       ...
       'Option Style', 'Package indicator', 'Package transaction price',
       'Package transaction price currency',
       'Package transaction price notation', 'Package transaction spread',
       'Package transaction spread currency',
       'Package transaction spread notation',
       'Physical delivery location-Leg 1', 'Delivery Type'],
      dtype='object', length=111)

In [25]:
data_type ='Commodities'
ticker = 'CFTC_CUMULATIVE_COMMODITIES_2023_07_25'


In [33]:
def point_crt(data_type, ticker, df, timestamp):
    field_list = df.columns.tolist()
    points = []
    
    for index, row in df.iterrows():
        timestamp = pd.Timestamp(row['Effective Date']).to_pydatetime()
        point_builder = Point(data_type).tag("ticker", ticker).time(timestamp)
        
        for field_name in field_list:
            point_builder = point_builder.field(field_name, row[field_name])
        
        points.append(point_builder)
    
    return points

In [34]:
def write_to_influxdb(df, ticker, bucket, org):
    # Initialize the InfluxDB client
    client = InfluxDBClient(url=url, token=token)

    # Create a WriteAPI instance
    write_api = client.write_api(write_options=SYNCHRONOUS)
    points = point_crt(data_type, ticker, df, timestamp)
    for point in points:
        write_api.write(bucket=bucket, org=org, record=point)
        # time.sleep(0.1)  # Optional: Sleep between each point to avoid rate-limiting
        print('done', ticker, 'for', timestamp)
    
    # Close the InfluxDB client
    client.close()

In [35]:
write_to_influxdb(df, ticker, bucket, org)

done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 2023-07-25 00:00:00
done CFTC_CUMULATIVE_COMMODITIES_2023_07_25 for 

### Mongodb

In [73]:
from pymongo import MongoClient

# Replace with your MongoDB server's IP address, port, username, and password
mongo_host = '3.109.41.82'
mongo_port = 27017
mongo_username = 'admin'
mongo_password = '123456789'

# Create a MongoDB client with authentication
client = MongoClient(
    host=mongo_host,
    port=mongo_port,
    username=mongo_username,
    password=mongo_password
)

In [2]:
print (client)

MongoClient(host=['3.109.41.82:27017'], document_class=dict, tz_aware=False, connect=True)


In [11]:
db = client.one_db

In [21]:
collection = db.one_coll

In [15]:
data = {"name": "John", "age": 30}
inserted_document = collection.insert_one(data)


In [6]:
inserted_document.inserted_id

ObjectId('64dc984ee9f68407b52456fd')

In [35]:
client.list_database_names()

['admin', 'config', 'ecommerce', 'local', 'testdb']

In [17]:
client.drop_database('one_db')

In [22]:
db=client.testdb
collection = db.one_coll
data = {"name": "John", "age": 30}
inserted_document = collection.insert_one(data)

In [24]:
db = client.ecommerce

# Create a collection called 'electronics'
electronics_collection = db.electronics

# List of product data to insert
products = [
    {
        "name": "Smartphone",
        "price": 499.99,
        "description": "High-end smartphone with advanced features."
    },
    {
        "name": "Laptop",
        "price": 1299.99,
        "description": "Powerful laptop for professional tasks."
    },
    {
        "name": "Smart TV",
        "price": 799.99,
        "description": "Large smart TV with 4K resolution."
    },
    {
        "name": "Headphones",
        "price": 149.99,
        "description": "Noise-canceling headphones for immersive audio."
    },
    {
        "name": "Tablet",
        "price": 299.99,
        "description": "Portable tablet for entertainment and productivity."
    }
]

# Insert the list of products into the 'electronics' collection
result = electronics_collection.insert_many(products)


In [26]:
db = client.ecommerce

# Access the 'electronics' collection
electronics_collection = db.electronics

# Retrieve all documents from the 'electronics' collection
all_products = electronics_collection.find()

# Print the retrieved documents
for product in all_products:
    print(product)

{'_id': ObjectId('64dc9a42e9f68407b5245700'), 'name': 'Smartphone', 'price': 499.99, 'description': 'High-end smartphone with advanced features.'}
{'_id': ObjectId('64dc9a42e9f68407b5245701'), 'name': 'Laptop', 'price': 1299.99, 'description': 'Powerful laptop for professional tasks.'}
{'_id': ObjectId('64dc9a42e9f68407b5245702'), 'name': 'Smart TV', 'price': 799.99, 'description': 'Large smart TV with 4K resolution.'}
{'_id': ObjectId('64dc9a42e9f68407b5245703'), 'name': 'Headphones', 'price': 149.99, 'description': 'Noise-canceling headphones for immersive audio.'}
{'_id': ObjectId('64dc9a42e9f68407b5245704'), 'name': 'Tablet', 'price': 299.99, 'description': 'Portable tablet for entertainment and productivity.'}


In [28]:
a=electronics_collection.find({'name' : 'Smart TV'})

In [30]:
for x in a:
    print (x)

{'_id': ObjectId('64dc9a42e9f68407b5245702'), 'name': 'Smart TV', 'price': 799.99, 'description': 'Large smart TV with 4K resolution.'}


In [31]:
def insert_dataframe_to_mongodb(dataframe, database_name, collection_name):
    # Create a MongoDB client with authentication
    client = MongoClient(
        host=mongo_host,
        port=mongo_port,
        username=mongo_username,
        password=mongo_password
    )
    
    # Access the specified database
    db = client[database_name]
    
    # Access the specified collection
    collection = db[collection_name]
    
    # Convert the DataFrame to a list of dictionaries
    records = dataframe.to_dict(orient='records')
    
    # Insert the list of dictionaries into the collection
    result = collection.insert_many(records)
    return result

In [37]:
db=client.testdb
coll = db.bag

In [43]:
insert_dataframe_to_mongodb(df,'testdb','bag1')

In [10]:
def get_dataframe_from_mongodb(database_name, collection_name):
    # Create a MongoDB client with authentication
    client = MongoClient(
        host=mongo_host,
        port=mongo_port,
        username=mongo_username,
        password=mongo_password
    )
    
    # Access the specified database
    db = client[database_name]
    
    # Access the specified collection
    collection = db[collection_name]
    
    # Retrieve all documents from the collection
    cursor = collection.find()
    
    # Convert the cursor to a list of dictionaries
    records = list(cursor)
    
    # Create a DataFrame from the list of dictionaries
    dataframe = pd.DataFrame(records)
    
    return dataframe

In [11]:
dt=get_dataframe_from_mongodb('testdb','bag1')
dt

/home/mehedi/anaconda3/envs/qs/lib/python3.10/site-packages/bson/__init__.py:1117: RuntimeWarning: coroutine 'main' was never awaited
  return _decode_all(data, opts)  # type:ignore[arg-type]
/home/mehedi/anaconda3/envs/qs/lib/python3.10/site-packages/bson/__init__.py:1117: RuntimeWarning: coroutine 'get_dataframe_from_mongodb_async' was never awaited
  return _decode_all(data, opts)  # type:ignore[arg-type]


In [1]:
import os
import pandas as pd
from datetime import datetime

In [42]:


def read_csvs_with_dates(folder_path, processed_files=set()):
    dataframes = []  # List to store DataFrames
    
    # Iterate through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv") and filename not in processed_files:
            print(filename)
            file_path = os.path.join(folder_path, filename)
            
            # Extract the date from the file name
            day_str = filename.split("_")[-1].replace(".csv", "")
            mon_str=filename.split("_")[-2].replace(".csv", "")
            yr_str=filename.split("_")[-3].replace(".csv", "")
            date_str = yr_str+'-'+mon_str+'-'+day_str
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Add a "date" column with the extracted date
            df.insert(0, "date", date_obj)
            
            # Append the DataFrame to the list
            dataframes.append(df)
            
            # Add the filename to the set of processed files
            processed_files.add(filename)
    
    return dataframes, processed_files  # Return the list of separate DataFrames and updated processed files set




In [40]:
folder_path = '/home/mehedi/Documents/docker_influxdb/DTCC_data/extracted_files/Forex'

In [44]:
processed_files = set()  # Initialize an empty set to store processed filenames
resulting_dataframes, processed_files = read_csvs_with_dates(folder_path, processed_files)

CFTC_CUMULATIVE_FOREX_2023_08_03.csv
CFTC_CUMULATIVE_FOREX_2023_07_28.csv


/tmp/ipykernel_43909/4064189164.py:18: DtypeWarning: Columns (33,39,40,43,76,100,107) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_43909/4064189164.py:18: DtypeWarning: Columns (33,40,107) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CFTC_CUMULATIVE_FOREX_2023_07_25.csv
CFTC_CUMULATIVE_FOREX_2023_08_09.csv


/tmp/ipykernel_43909/4064189164.py:18: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_43909/4064189164.py:18: DtypeWarning: Columns (16,33,39,40,43,62,69,72,73,92,103,107) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CFTC_CUMULATIVE_FOREX_2023_07_24.csv
CFTC_CUMULATIVE_FOREX_2023_08_12.csv
CFTC_CUMULATIVE_FOREX_2023_08_07.csv


...

CFTC_CUMULATIVE_FOREX_2023_08_02.csv


/tmp/ipykernel_43909/4064189164.py:18: DtypeWarning: Columns (9,13,14,16,17,33,34,35,36,39,40,41,42,43,44,45,46,49,52,53,60,61,62,68,69,72,73,76,85,86,88,89,90,92,100,101,102,103,104,105,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CFTC_CUMULATIVE_FOREX_2023_08_14.csv
CFTC_CUMULATIVE_FOREX_2023_08_11.csv


/tmp/ipykernel_43909/4064189164.py:18: DtypeWarning: Columns (14,16,33,34,39,40,60,62,69,72,73,90,92,100,104,105,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_43909/4064189164.py:18: DtypeWarning: Columns (33,40,100) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CFTC_CUMULATIVE_FOREX_2023_07_22.csv


In [59]:
for df in resulting_dataframes:
    print(df)

            date  Dissemination Identifier  Original Dissemination Identifier  \
0     2023-07-18                 691462457                                NaN   
1     2023-07-18                 691465719                                NaN   
2     2023-07-18                 691465720                        691465712.0   
3     2023-07-18                 691477539                                NaN   
4     2023-07-18                 691477548                                NaN   
...          ...                       ...                                ...   
28876 2023-07-18                 691454090                                NaN   
28877 2023-07-18                 691454099                                NaN   
28878 2023-07-18                 691454112                                NaN   
28879 2023-07-18                 691454125                                NaN   
28880 2023-07-18                 691454145                                NaN   

      Action type Event typ

In [1]:
import function as f

In [2]:
database_name='test_db'
collection_name = 'bag'

In [3]:
f.get_dataframe_from_mongodb(database_name,collection_name,target_date='2023-08-03')

,_id,date,Dissemination Identifier,Original Dissemination Identifier,Action type,Event type,Event timestamp,Amendment indicator,Asset Class,Product name,...,Option Style,Package indicator,Package transaction price,Package transaction price currency,Package transaction price notation,Package transaction spread,Package transaction spread currency,Package transaction spread notation,Physical delivery location-Leg 1,Delivery Type
0,64dcd009a179408b8c4fe38e,2023-08-03,707172737,NaN,NEWT,TRAD,2023-08-03T02:43:17Z,NaN,FX,ForeignExchange:NDF,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,64dcd009a179408b8c4fe38f,2023-08-03,707171524,NaN,NEWT,TRAD,2023-08-03T02:13:43Z,NaN,FX,ForeignExchange:NDF,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,64dcd009a179408b8c4fe390,2023-08-03,707171525,NaN,NEWT,TRAD,2023-08-03T02:13:43Z,NaN,FX,ForeignExchange:NDF,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,64dcd009a179408b8c4fe391,2023-08-03,707172763,NaN,NEWT,TRAD,2023-08-03T02:42:31Z,NaN,FX,ForeignExchange:NDF,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,64dcd009a179408b8c4fe392,2023-08-03,707171533,NaN,NEWT,TRAD,2023-08-03T02:13:51Z,NaN,FX,ForeignExchange:NDF,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33526,64dcd009a179408b8c506684,2023-08-03,708347744,677310046.0,MODI,TRAD,2023-08-03T23:50:45Z,True,FX,ForeignExchange:NDF,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Physical
33527,64dcd009a179408b8c506685,2023-08-03,708347745,677318448.0,MODI,TRAD,2023-08-03T23:50:48Z,True,FX,ForeignExchange:NDF,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Physical
33528,64dcd009a179408b8c506686,2023-08-03,708347778,NaN,NEWT,TRAD,2023-08-03T23:51:44Z,NaN,FX,ForeignExchange:Forward,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Physical
33529,64dcd009a179408b8c506687,2023-08-03,708347808,707932401.0,EROR,NaN,2023-08-03T23:57:54Z,NaN,FX,ForeignExchange:VanillaOption,...,EUROPEAN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Physical


In [75]:
client.list_database_names()

['admin', 'config', 'ecommerce', 'local', 'test_db', 'testdb']

In [76]:
db=client.test_db

In [78]:
coll=db.bag

In [82]:
coll.find_one()

{'_id': ObjectId('64dcd009a179408b8c4fe38e'),
 'date': datetime.datetime(2023, 8, 3, 0, 0),
 'Dissemination Identifier': 707172737,
 'Original Dissemination Identifier': nan,
 'Action type': 'NEWT',
 'Event type': 'TRAD',
 'Event timestamp': '2023-08-03T02:43:17Z',
 'Amendment indicator': nan,
 'Asset Class': 'FX',
 'Product name': 'ForeignExchange:NDF',
 'Cleared': 'N',
 'Mandatory clearing indicator': False,
 'Execution Timestamp': '2023-08-03T02:43:17Z',
 'Effective Date': '2023-08-03',
 'Expiration Date': '2023-09-07',
 'Maturity date of the underlier': nan,
 'Non-standardized term indicator': False,
 'Platform identifier': 'XXXX',
 'Prime brokerage transaction indicator': False,
 'Block trade election indicator': False,
 'Large notional off-facility swap election indicator': False,
 'Notional amount-Leg 1': '1,000,000',
 'Notional amount-Leg 2': '1,296,860,000',
 'Notional currency-Leg 1': 'USD',
 'Notional currency-Leg 2': 'KRW',
 'Notional quantity-Leg 1': nan,
 'Notional quanti